In [2]:
import pandas as pd
import pymongo
import os
from pymongo import MongoClient
import pprint

In [3]:
client = MongoClient(
    host = "127.0.0.1",
    port = 27017,
    username = "antoine",
    password = "pela"
)

print(client.list_database_names())

['admin', 'config', 'local', 'reco_movies']


In [4]:
db = client['reco_movies']

In [9]:
# Lister et afficher les collections
collections = db.list_collection_names()
print("Collections dans la base de données :")
for collection in collections:
    print(collection)

Collections dans la base de données :
ratings


### CHARGEMENT DES DONNEES DANS NOS COLLECTIONS

In [10]:
# CREATION COLLECTION MOVIES
collection_movies = db['movies']

movies = pd.read_csv('/home/antoine/Ml_Ops_Movies_Reco/src/data/data/raw/movies.csv')

# Split sur les pipes
movies['genres'] = movies['genres'].apply(lambda x: x.split("|"))
# Extraction de l'année et mise à jour du titre
movies['year'] = movies['title'].str.extract(r'\((\d{4})\)')[0]
movies['title'] = movies['title'].str.replace(r' \(\d{4}\)', '', regex=True)
movies.ffill(inplace= True)
movies.head()


,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [11]:
records_movies =movies.to_dict(orient="records")

collection_movies.insert_many(records_movies)

InsertManyResult([ObjectId('67211c9660d592dc99f8e922'), ObjectId('67211c9660d592dc99f8e923'), ObjectId('67211c9660d592dc99f8e924'), ObjectId('67211c9660d592dc99f8e925'), ObjectId('67211c9660d592dc99f8e926'), ObjectId('67211c9660d592dc99f8e927'), ObjectId('67211c9660d592dc99f8e928'), ObjectId('67211c9660d592dc99f8e929'), ObjectId('67211c9660d592dc99f8e92a'), ObjectId('67211c9660d592dc99f8e92b'), ObjectId('67211c9660d592dc99f8e92c'), ObjectId('67211c9660d592dc99f8e92d'), ObjectId('67211c9660d592dc99f8e92e'), ObjectId('67211c9660d592dc99f8e92f'), ObjectId('67211c9660d592dc99f8e930'), ObjectId('67211c9660d592dc99f8e931'), ObjectId('67211c9660d592dc99f8e932'), ObjectId('67211c9660d592dc99f8e933'), ObjectId('67211c9660d592dc99f8e934'), ObjectId('67211c9660d592dc99f8e935'), ObjectId('67211c9660d592dc99f8e936'), ObjectId('67211c9660d592dc99f8e937'), ObjectId('67211c9660d592dc99f8e938'), ObjectId('67211c9660d592dc99f8e939'), ObjectId('67211c9660d592dc99f8e93a'), ObjectId('67211c9660d592dc99f8e9

In [ ]:
# CREATION COLLECTION RATINGS
collection_ratings = db['ratings']

ratings = pd.read_csv('/home/antoine/Ml_Ops_Movies_Reco/src/data/data/raw/ratings.csv')

ratings = ratings.drop('timestamp', axis = 1)

records_ratings = ratings.to_dict(orient="records")

collection_ratings.insert_many(records_ratings)

In [12]:
# CREATION COLLECTION LINKS
collection_links = db['links']

links = pd.read_csv('/home/antoine/Ml_Ops_Movies_Reco/app/fastapi/app/raw/links2.csv', index_col=0)

links = links.drop('tmdbId', axis = 1)

records_links = links.to_dict(orient="records")

collection_links.insert_many(records_links)

InsertManyResult([ObjectId('67211ca060d592dc99f953b0'), ObjectId('67211ca060d592dc99f953b1'), ObjectId('67211ca060d592dc99f953b2'), ObjectId('67211ca060d592dc99f953b3'), ObjectId('67211ca060d592dc99f953b4'), ObjectId('67211ca060d592dc99f953b5'), ObjectId('67211ca060d592dc99f953b6'), ObjectId('67211ca060d592dc99f953b7'), ObjectId('67211ca060d592dc99f953b8'), ObjectId('67211ca060d592dc99f953b9'), ObjectId('67211ca060d592dc99f953ba'), ObjectId('67211ca060d592dc99f953bb'), ObjectId('67211ca060d592dc99f953bc'), ObjectId('67211ca060d592dc99f953bd'), ObjectId('67211ca060d592dc99f953be'), ObjectId('67211ca060d592dc99f953bf'), ObjectId('67211ca060d592dc99f953c0'), ObjectId('67211ca060d592dc99f953c1'), ObjectId('67211ca060d592dc99f953c2'), ObjectId('67211ca060d592dc99f953c3'), ObjectId('67211ca060d592dc99f953c4'), ObjectId('67211ca060d592dc99f953c5'), ObjectId('67211ca060d592dc99f953c6'), ObjectId('67211ca060d592dc99f953c7'), ObjectId('67211ca060d592dc99f953c8'), ObjectId('67211ca060d592dc99f953

### VISUALISATION DES COLLECTIONS

In [13]:
ratings = db['ratings']

pprint.pprint(ratings.find_one())

{'_id': ObjectId('671a9b3b0f9ef42264412d63'),
 'movieId': 2,
 'rating': 3.5,
 'userId': 1}


In [14]:
# Compter le nombre total de documents
nombre_documents = ratings.count_documents({})
print("Nombre total de documents :", nombre_documents)

Nombre total de documents : 20300263


In [15]:
movies = db.movies

pprint.pprint(movies.find_one())

{'_id': ObjectId('67211c9660d592dc99f8e922'),
 'genres': ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 'movieId': 1,
 'title': 'Toy Story',
 'year': '1995'}


In [16]:
# Compter le nombre total de documents
nombre_movies = movies.count_documents({})
print("Nombre total de movies :", nombre_movies)

Nombre total de movies : 27278


In [17]:
links = db.links

pprint.pprint(links.find_one())

{'_id': ObjectId('67211ca060d592dc99f953b0'),
 'cover_link': 'https://m.media-amazon.com/images/M/MV5BMDU2ZWJlMjktMTRhMy00ZTA5LWEzNDgtYmNmZTEwZTViZWJkXkEyXkFqcGdeQXVyNDQ2OTk4MzI@._V1_QL75_UX190_CR0,1,190,281_.jpg',
 'imdbId': 114709,
 'movieId': 1}


### EXTRACTION DES DONNEES

In [22]:
# Récupérer le movieId maximum dans la collection links
max_movie_id_links = links.find().sort("movieId", pymongo.DESCENDING).limit(1)

# Extraire le movieId
for doc in max_movie_id_links:
    max_movie_id = doc['movieId']
    print(f"Le movieId maximum est : {max_movie_id}")

Le movieId maximum est : 131262
